<a href="https://colab.research.google.com/github/PhuongAnhMannheim/-Team-Project-IPDW/blob/master/closed_feature_extraction_overlap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Schema Mapping using closed feature extraction (FE)

##Set up the environment


In [1]:
# import required packages
import numpy as np
import pandas as pd
import math
import json
from google.colab import files
import gspread
import xlrd


# access Google Drive via Google Cloud SDK
!pip install -U -q PyDrive
!pip install --upgrade -q gspread
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os
import pandas as pd
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# authorization for the gold standard file
gc = gspread.authorize(GoogleCredentials.get_application_default())


    100% |████████████████████████████████| 993kB 18.8MB/s 


## Loading the data sources

###dogfood file -> dogfood_title | dogfood_description


In [6]:
# identifing source file
listed = drive.ListFile({'q': "title contains 'dogfood_all_offers_Jan7.csv'"}).GetList()
for file in listed:
  print('title {}, id {}'.format(file['title'], file['id']))

download_path = os.path.expanduser('~/data')
try:
  os.makedirs(download_path)
except FileExistsError:
  pass

dogfood_file = os.path.join(download_path, 'dogfood_all_offers_Jan7.csv')
temp_file = drive.CreateFile({'id': '1wbJvxAbMDoYzyWAjXk52MUCZbypezRFP'})
temp_file.GetContentFile(dogfood_file)

# loading the file into pandas
dogfood_title_data = pd.read_csv(dogfood_file,sep=',')
dogfood_description_data = pd.read_csv(dogfood_file,sep=',')

dogfood_title_data = dogfood_title_data.drop(['clusterID','clusterSize','brand','weightkg ','description','id1','id2','id3','price','priceCurrency','filename'], axis=1)
print('Dogfood Title Data Preview: \n', dogfood_title_data.head(), '\n')
print('Statistics to Dogfood Title Data: \n', dogfood_title_data.describe(), '\n')

dogfood_description_data = dogfood_description_data.drop(['clusterID','clusterSize','name','brand','weightkg ','id1','id2','id3','price','priceCurrency','filename'], axis=1)
print('Dogfood Description Data Preview: \n', dogfood_description_data.head(), '\n')
print('Statistics to Dogfood Description Data: \n', dogfood_description_data.describe())


title dogfood_all_offers_Jan7.csv, id 1wbJvxAbMDoYzyWAjXk52MUCZbypezRFP
Dogfood Title Data Preview: 
       offerID                                               name
0  bibita_001       hills prescription diet canine jd joint care
1  bibita_002         royal canin giant starter - mother babydog
2  bibita_003  hills prescription diet canine dd food sensiti...
3  bibita_004  hills prescription diet canine zd food sensiti...
4  bibita_005     hills prescription diet canine ud urinary care 

Statistics to Dogfood Title Data: 
                   offerID                                               name
count                3265                                               3265
unique               3265                                               1833
top     hardwarestore_016  taste wild grain free appalachian valley small...
freq                    1                                                  8 

Dogfood Description Data Preview: 
       offerID                                  

###laptop file -> laptop_title | laptop_description

In [7]:
# get identifier of source file
listed = drive.ListFile({'q': "title contains 'Laptop_all_offers_Jan6.csv'"}).GetList()
for file in listed:
  print('title {}, id {}'.format(file['title'], file['id']))

## download file
download_path = os.path.expanduser('~/data')
try:
  os.makedirs(download_path)
except FileExistsError:
  pass

laptop_file = os.path.join(download_path, 'Laptop_all_offers_Jan6.csv')
temp_file = drive.CreateFile({'id': '18M-bymvLXIbhmN5xCg36ilYgfbozftVq'})
temp_file.GetContentFile(laptop_file)

##loading the file into pandas
laptop_title_data = pd.read_csv(laptop_file,sep=',')
laptop_description_data = pd.read_csv(laptop_file,sep=',')

laptop_title_data = laptop_title_data.drop(['clusterid','size','id1','id2','id3','brand','description','price','currency'], axis=1)
print('Laptop Title Data Preview: \n', laptop_title_data.head(), '\n')
print('Statistics to Laptop Title Data: \n', laptop_title_data.describe(), '\n')

laptop_description_data = laptop_description_data.drop(['clusterid','size','id1','id2','id3','productname','brand','price','currency'], axis=1)
print('Laptop Description Data Preview: \n', laptop_description_data.head(), '\n')
print('Statistics to Laptop Title Data: \n', laptop_description_data.describe(), '\n')



title laptop_all_offers_jan6.csv, id 1RkCiWck4kXAVFLthaBa6R4sVXRhPgYwy
title Laptop_all_offers_Jan6.csv, id 18M-bymvLXIbhmN5xCg36ilYgfbozftVq
Laptop Title Data Preview: 
            productid                                        productname
0     microcart_0161  zeepad 8in 1gb 8gb qc zeepad x8 wht 0852170006...
1      sdicraft_0604     lenovo 8 tab 4 8 16gb tablet wi fi slate black
2  bhphotovideo_0001     116 32gb flex 11 multi touch 2 in 1 chromebook
3    truecurate_0345  lenovo za270025us 2 in 1 116ampquot touch scre...
4       macmall_0344  lenovo n23 yoga chromebook mediatek mtk 8173c ... 

Statistics to Laptop Title Data: 
              productid                                productname
count            12710                                      12710
unique           12710                                      10599
top     techcrave_1035  panasonic toughbook 54 gloved multi touch
freq                 1                                         69 

Laptop Description Data Prev

## Building the dictionary

### Dogfood Dictionary


- Dictionary with multiple values for each key


*   For the prodecure on obtaining the dictionary content the filter options of each website have been observed
*   practical: in order to make sure that we are using the same dictionary for all subgroups, we are refering to one central dictionary file from the google drive


In [2]:
# Open our new sheet and read some data.
dogfood_dict_worksheet = gc.open('Dictionary_Dogfood').sheet1

#Analyzing spreadsheet
print('maximal number of keys = ', dogfood_dict_worksheet.col_count)
print('maximal number of values = ', dogfood_dict_worksheet.row_count)
print('amount of values for lifestage=', len(dogfood_dict_worksheet.col_values(2))-1)

dogfood_list_attr = dogfood_dict_worksheet.row_values(1)
print('keys = ', dogfood_list_attr)


dogfood_dict= {}
for i in range (0, dogfood_dict_worksheet.col_count):
  lifestage_values = dogfood_dict_worksheet.col_values(i+1)
  del lifestage_values[0]
  dogfood_dict.update({dogfood_list_attr[i] : lifestage_values} )
print('dictionary = ', dogfood_dict)

maximal number of keys =  10
maximal number of values =  176
amount of values for lifestage= 9
keys =  ['Brand', 'Lifestage/ Ranges/ Dog Age', 'Breed Size/ Ranges/ Dog Size', 'Food/ Type', 'Flavor', 'Weights', 'Weight Unit of Measurement', 'breed', 'price', 'priceCurrency']
dictionary =  {'Brand': ['addiction', 'adirondack', 'against the grain', 'almo nature', 'alpo', 'american journey', 'american natural premium', 'annamaet', 'artemis', 'authority', 'avoderm', 'basics', 'beaverdam pet food', 'beneful', 'bil jac', 'black gold', 'blackwood', 'blue basics', 'blue buffalo', 'blue freedom', 'blue life protection', 'blue natural veterinary diet', 'blue wilderness', 'bravo', 'breeders choice', 'brothers complete', 'buckley', 'canada fresh', 'canidae', 'canine caviar', 'caru', 'castor  pollux', 'cat man doo', 'cesar', 'chicken soup', 'cloud star', 'crave', 'daves pet food', 'diamond', 'dog chow', 'dog lovers gold', 'dogswell', 'dr garys best breed', 'dr harveys', 'dr tims', 'eagle pack', 'ear

### Laptop Dictionary

In [5]:
# Open our new sheet and read some data.
laptop_dict_worksheet = gc.open('Dictionary_Laptop').sheet1

#Analyzing the dictionary
print('maximal number of keys = ', laptop_dict_worksheet.col_count)
print('maximal number of values = ', laptop_dict_worksheet.row_count)


laptop_list_attr = laptop_dict_worksheet.row_values(1)
print('keys = ', laptop_list_attr)


laptop_dict= {}
for i in range (0, laptop_dict_worksheet.col_count):
  lifestage_values = laptop_dict_worksheet.col_values(i+1)
  del lifestage_values[0]
  laptop_dict.update({laptop_list_attr[i] : lifestage_values} )

print('dictionary = ', laptop_dict)

maximal number of keys =  11
maximal number of values =  43
keys =  ['Type', 'Brand', 'Operating System', 'RAM', 'Screen Size', 'Graphics Card ', 'Hard drive type', 'HDD Size', 'SSD Size', 'Processor Model', 'Processor Brand']
dictionary =  {'Type': ['2 in 1 laptops', 'touchscreen laptops', 'gaming laptops', 'ultrabook laptops', 'laptops', 'notebooks', 'ultrabook', 'workstations', 'notebook', 'mobile workstation', 'gaming notebook', 'netbook', 'mobile workstation ultrabook', '2 in 1 ultrabook', '2 in 1 mobile workstation', 'thin client notebook'], 'Brand': ['hp laptops', 'dell laptops', 'lenovo laptops', 'asus laptops', 'samsung laptops', 'alienware laptops', 'hp omen laptops', 'acer laptops', 'toshiba laptops', 'surface laptops', 'msi laptops', 'microsoft', 'dell', 'hp', 'lenovo', 'asus', 'acer', 'samsung', 'msi', 'alienware', 'lg', 'razer', 'toshiba', 'hyundai', 'wacom', 'hp omen', 'asus', 'dell computer', 'google', 'huawei', 'lenovo', 'lg', 'panasonic', 'samsung', 'toshiba  acer', '

##Building extraction functionalities

One possibility to provide the dictionary based feature extraction is overlap using tokenization and intersection.

In [0]:
    def intersection(lst1, lst2):
        return list(set(lst1) & set(lst2))

###For Dogfood

In [10]:
class Dogfood_Overlap_shortest():
  
    def __init__(self,list_content,list_id,list_attr,attr_select):
        self.list_content=list_content
        self.list_id=list_id
        self.list_attr=list_attr
        self.attr_select=attr_select
    
    def get_attribute(self):
        dict_item = dict()
        for i in range(len(self.list_id)):
          list_row = list()
          str_single = self.list_content[i]
          for pos in range(len(self.list_attr)):
            attr = self.get_generic(str_single, list_attr[pos])
            list_row.append(attr)
          dict_item[self.list_id[i]] = list_row

        return dict_item
      
    def get_generic(self, str_single, dict_elem):
      str_single=str(str_single)
      bow = str_single.split(' ')
      str_sub = []
      for ws in range(5):
        for i in range(len(bow)):
          str_sub.append(" ".join(bow[i:i+ws]))
       
      str_attr = intersection(str_sub, dogfood_dict[dict_elem])
      if len(str_attr) > 0:
        return str_attr[0]
   
  
class Dogfood_Overlap_longest():
  
    def __init__(self,list_content,list_id,list_attr,attr_select):
        self.list_content=list_content
        self.list_id=list_id
        self.list_attr=list_attr
        self.attr_select=attr_select
    
    def get_attribute(self):
        dict_item = dict()
        for i in range(len(self.list_id)):
          list_row = list()
          str_single = self.list_content[i]
          for pos in range(len(self.list_attr)):
            attr = self.get_generic(str_single, list_attr[pos])
            list_row.append(attr)
          dict_item[self.list_id[i]] = list_row

        return dict_item
      
    def get_generic(self, str_single, dict_elem):
      str_single = str(str_single)
      bow = str_single.split(' ')
      str_sub = []
      for ws in range(5):
        for i in range(len(bow)):
          str_sub.append(" ".join(bow[i:(i + 4 - ws)]))
       
      str_attr = intersection(str_sub, dogfood_dict[dict_elem])
      if len(str_attr) > 0:
        return str_attr[0]
      
      
#try-out of overlap function      
if __name__=="__main__":
    str1 = "taste of the wild high prairie canine adult"
    str2 = "hills science diet adult 11+ small toy breed age defying chicken meal rice  barley recipe dry dog food case of 20 4.5 kg bag"
    str3 = "royal canin labrador retriever adult sterilised"
    list1 = [str1, str2, str3]
    list2 = ["str1","str2","str3"]
    list_attr = dogfood_list_attr
    rd=Dogfood_Overlap_longest(list1,list2,dogfood_list_attr,dogfood_dict)
    dict_item=rd.get_attribute()
    print(dict_item) 

{'str1': ['taste of the wild', 'adult', None, None, None, None, None, None, None, None], 'str2': ['hills', 'adult', 'small', 'dry', 'chicken', None, 'kg', None, None, None], 'str3': ['royal canin', 'adult', None, None, None, None, None, 'retriever', None, None]}


###For Laptop

In [11]:
class Laptop_Overlap_shortest():
  
    def __init__(self,list_content,list_id,list_attr,attr_select):
        self.list_content=list_content
        self.list_id=list_id
        self.list_attr=list_attr
        self.attr_select=attr_select
    
    def get_attribute(self):
        dict_item = dict()
        for i in range(len(self.list_id)):
          list_row = list()
          str_single = self.list_content[i]
          for pos in range(len(self.list_attr)):
            attr = self.get_generic(str_single, list_attr[pos])
            list_row.append(attr)
          dict_item[self.list_id[i]] = list_row

        return dict_item
      
    def get_generic(self, str_single, dict_elem):
      str_single=str(str_single)
      bow = str_single.split(' ')
      str_sub = []
      for ws in range(5):
        for i in range(len(bow)):
          str_sub.append(" ".join(bow[i:i+ws]))
       
      str_attr = intersection(str_sub, laptop_dict[dict_elem])
      if len(str_attr) > 0:
        return str_attr[0]
      
class Laptop_Overlap_longest():
  
    def __init__(self,list_content,list_id,list_attr,attr_select):
        self.list_content=list_content
        self.list_id=list_id
        self.list_attr=list_attr
        self.attr_select=attr_select
    
    def get_attribute(self):
        dict_item = dict()
        for i in range(len(self.list_id)):
          list_row = list()
          str_single = self.list_content[i]
          for pos in range(len(self.list_attr)):
            attr = self.get_generic(str_single, list_attr[pos])
            list_row.append(attr)
          dict_item[self.list_id[i]] = list_row

        return dict_item
      
    def get_generic(self, str_single, dict_elem):
      str_single=str(str_single)
      bow = str_single.split(' ')
      str_sub = []
      for ws in range(5):
        for i in range(len(bow)):
          str_sub.append(" ".join(bow[i:(i + 4 - ws)]))
       
      str_attr = intersection(str_sub, laptop_dict[dict_elem])
      if len(str_attr) > 0:
        return str_attr[0]
      
if __name__=="__main__":
    str1 = "medion akoya 20ghz s3409 md 60219 337 ddr3 14 cm 133 gtx 750ti zoll mattes full hd display notebook intel core i5 7200u 8gb ram 256gb ssd intel hd grafik win 10 home silber"
    str2 = "asus vivobook full hd laptop intel i7 7500u 27ghz 8gb ram 128gb solid state drive 1tb hdd windows 10 fingerprint sensor156s510ua db71"
    str3=  "dell xps 9000 13 9370 hd 9370 500gb i9 1024x768 "
    str4=  "116 32gb chromebook 11 3180 r7 2700u 13.3"
    list1 = [str1, str2,str3,str4]
    list2 = ["str1","str2","str3","str4"]
    list_attr = laptop_list_attr
    rd=Laptop_Overlap_shortest(list1,list2,laptop_list_attr,laptop_dict)
    dict_item=rd.get_attribute()
    print(dict_item)

{'str1': ['notebook', None, None, '8gb', '133', None, None, '256gb', '8gb', 'core i5', 'intel'], 'str2': [None, 'asus', 'windows 10', '8gb', None, None, None, '128gb', '128gb', None, 'intel'], 'str3': [None, 'dell', None, None, '13', None, None, '500gb', '500gb', None, None], 'str4': [None, None, None, '32gb', '116', None, None, '32gb', '32gb', None, None]}


## Text feature extraction

###overlap_shortest for dogfood title


In [12]:
dogfood_list_attr

['Brand',
 'Lifestage/ Ranges/ Dog Age',
 'Breed Size/ Ranges/ Dog Size',
 'Food/ Type',
 'Flavor',
 'Weights',
 'Weight Unit of Measurement',
 'breed',
 'price',
 'priceCurrency']

In [0]:


offer_id=dogfood_title_data['offerID']
title=dogfood_title_data['name']
rd=Dogfood_Overlap_shortest(title,offer_id,dogfood_list_attr,dogfood_dict)
dict_item=rd.get_attribute()
dict_item

{'bibita_001': ['hills prescription diet',
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 'bibita_002': ['royal canin', None, 'giant', None, None, None, None, None],
 'bibita_003': ['hills prescription diet',
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 'bibita_004': ['hills prescription diet',
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 'bibita_005': ['hills prescription diet',
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 'bibita_006': ['royal canin', 'junior', None, None, None, None, None, None],
 'bibita_007': ['hills prescription diet',
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 'bibita_008': ['royal canin', 'junior', None, None, None, None, None, None],
 'bibita_009': ['royal canin', None, 'maxi', None, None, None, None, None],
 'bibita_010': ['hills prescription diet',
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 'bibita_011': ['hills prescription diet',
  None,
  None,
  None,
  None,
  None,
  None,


In [0]:
offer_id=dogfood_title_data['offerID']
title=dogfood_title_data['name']
rd=Dogfood_Overlap_shortest(title,offer_id,dogfood_list_attr,dogfood_dict)
dict_item=rd.get_attribute()
dict_item

df_attr_dogfood_overlap_shortest=pd.DataFrame.from_dict(dict_item,orient='index')

columns_dict={
    0:"brand",
    1:"lifestage",
    2:"breed_size",
    3:"food_type",
    4:"flavour",
    5:"weight",
    6:"breed",
    7:"price"
}
df_attr_dogfood_overlap_shortest=df_attr_dogfood_overlap_shortest.rename(columns=columns_dict)
df_attr_dogfood_overlap_shortest

,brand,lifestage,breed_size,food_type,flavour,weight,breed,price
bibita_001,hills prescription diet,None,None,None,None,None,None,None
bibita_002,royal canin,None,giant,None,None,None,None,None
bibita_003,hills prescription diet,None,None,None,None,None,None,None
bibita_004,hills prescription diet,None,None,None,None,None,None,None
bibita_005,hills prescription diet,None,None,None,None,None,None,None
bibita_006,royal canin,junior,None,None,None,None,None,None
bibita_007,hills prescription diet,None,None,None,None,None,None,None
bibita_008,royal canin,junior,None,None,None,None,None,None
bibita_009,royal canin,None,maxi,None,None,None,None,None
bibita_010,hills prescription diet,None,None,None,None,None,None,None


###overlap_longest for dogfood title

In [0]:
offer_id=dogfood_title_data['offerID']
title=dogfood_title_data['name']
rd=Dogfood_Overlap_longest(title,offer_id,dogfood_list_attr,dogfood_dict)
dict_item=rd.get_attribute()
dict_item

df_attr_dogfood_overlap_longest=pd.DataFrame.from_dict(dict_item,orient='index')
columns_dict={
    0:"brand",
    1:"lifestage",
    2:"breed_size",
    3:"food_type",
    4:"flavour",
    5:"weight",
    6:"breed",
    7: "price"
}
df_attr_dogfood_overlap_longest=df_attr_dogfood_overlap_longest.rename(columns=columns_dict)
df_attr_dogfood_overlap_longest

,brand,lifestage,breed_size,food_type,flavour,weight,breed,price
bibita_001,hills prescription diet,None,None,None,None,None,None,None
bibita_002,royal canin,None,giant,None,None,None,None,None
bibita_003,hills prescription diet,None,None,None,None,None,None,None
bibita_004,hills prescription diet,None,None,None,None,None,None,None
bibita_005,hills prescription diet,None,None,None,None,None,None,None
bibita_006,royal canin,junior,None,None,None,None,None,None
bibita_007,hills prescription diet,None,None,None,None,None,None,None
bibita_008,royal canin,junior,None,None,None,None,None,None
bibita_009,royal canin,None,maxi,None,None,None,None,None
bibita_010,hills prescription diet,None,None,None,None,None,None,None


###overlap_shortest for dogfood description


In [0]:
offer_id=dogfood_description_data['offerID']
title=dogfood_description_data['description']
rd=Dogfood_Overlap_shortest(title,offer_id,dogfood_list_attr,dogfood_dict)
dict_item=rd.get_attribute()
dict_item

df_attr_dogfood_descr_overlap_shortest=pd.DataFrame.from_dict(dict_item,orient='index')
columns_dict={
    0:"brand",
    1:"lifestage",
    2:"breed_size",
    3:"food_type",
    4:"flavour",
    5:"weight",
    6:"breed",
    7: "price"
}
df_attr_dogfood_descr_overlap_shortest=df_attr_dogfood_descr_overlap_shortest.rename(columns=columns_dict)
df_attr_dogfood_descr_overlap_shortest

,brand,lifestage,breed_size,food_type,flavour,weight,breed,price
bibita_001,hills prescription diet,senior,None,dry,chicken,None,None,None
bibita_002,royal canin,None,large,dry,None,None,None,None
bibita_003,hills prescription diet,None,None,None,duck,None,None,None
bibita_004,hills prescription diet,None,None,dry,liver,None,None,None
bibita_005,hills prescription diet,None,small,dry,None,None,None,None
bibita_006,royal canin,junior,small,dry,None,None,None,None
bibita_007,hills prescription diet,None,None,None,None,None,None,None
bibita_008,royal canin,adult,None,None,None,None,None,None
bibita_009,royal canin,adult,large,dry,None,None,None,None
bibita_010,hills,adult,None,dry,chicken,None,None,None


###overlap_longest for dogfood description

In [0]:
offer_id=dogfood_description_data['offerID']
title=dogfood_description_data['description']
rd=Dogfood_Overlap_longest(title,offer_id,dogfood_list_attr,dogfood_dict)
dict_item=rd.get_attribute()
dict_item

df_attr_dogfood_descr_overlap_longest=pd.DataFrame.from_dict(dict_item,orient='index')
columns_dict={
    0:"brand",
    1:"lifestage",
    2:"breed_size",
    3:"food_type",
    4:"flavour",
    5:"weight",
    6:"breed"
}
df_attr_dogfood_descr_overlap_longest=df_attr_dogfood_descr_overlap_longest.rename(columns=columns_dict)
df_attr_dogfood_descr_overlap_longest

,brand,lifestage,breed_size,food_type,flavour,weight,breed,7
bibita_001,hills prescription diet,senior,None,dry,chicken,None,None,None
bibita_002,royal canin,None,large,dry,None,None,None,None
bibita_003,hills prescription diet,None,None,None,duck,None,None,None
bibita_004,hills prescription diet,None,None,dry,liver,None,None,None
bibita_005,hills prescription diet,None,small,dry,None,None,None,None
bibita_006,royal canin,junior,small,dry,None,None,None,None
bibita_007,hills prescription diet,None,None,None,None,None,None,None
bibita_008,royal canin,adult,None,None,None,None,None,None
bibita_009,royal canin,adult,large,dry,None,None,None,None
bibita_010,hills,adult,None,dry,chicken,None,None,None


###overlap_shortest for laptop title


In [0]:
offer_id=laptop_title_data['productid']
title=laptop_title_data['productname']
list_attr = laptop_list_attr
rd=Laptop_Overlap_shortest(title,offer_id,laptop_list_attr,laptop_dict)
laptop_title_item=rd.get_attribute()
laptop_title_item

df_attr_laptop_overlap_shortest=pd.DataFrame.from_dict(laptop_title_item,orient='index')
columns_dict={
    0:"type",
    1:"brand",
    2:"os",
    3:"RAM",
    4:"screen_size",
    5:"graphic_card_brand",
    6:"hard_drive_type",
    7:"HDD_size",
    8:"SSD_size",
    9:"processor_model",
    10:"processor_brand"
}
df_attr_laptop_overlap_shortest = df_attr_laptop_overlap_shortest.rename(columns=columns_dict)
df_attr_laptop_overlap_shortest.head()

,type,brand,os,RAM,screen_size,graphic_card_brand,hard_drive_type,HDD_size,SSD_size,processor_model,processor_brand
microcart_0161,None,None,None,1gb,None,None,None,None,8gb,None,None
sdicraft_0604,None,lenovo,None,16gb,None,None,None,16gb,16gb,None,None
bhphotovideo_0001,None,None,None,32gb,None,None,None,32gb,32gb,None,None
truecurate_0345,None,lenovo,None,4gb,None,None,None,32gb,4gb,None,None
macmall_0344,notebook,None,None,32gb,None,None,None,32gb,32gb,None,mediatek


###overlap_longest for laptop title

In [0]:
offer_id=laptop_title_data['productid']
title=laptop_title_data['productname']
list_attr = laptop_list_attr
rd=Laptop_Overlap_longest(title,offer_id,laptop_list_attr,laptop_dict)
laptop_title_item=rd.get_attribute()
laptop_title_item

df_attr_laptop_overlap_longest=pd.DataFrame.from_dict(laptop_title_item,orient='index')
columns_dict={
    0:"type",
    1:"brand",
    2:"os",
    3:"RAM",
    4:"screen_size",
    5:"graphic_card_brand",
    6:"hard_drive_type",
    7:"HDD_size",
    8:"SSD_size",
    9:"processor_model",
    10:"processor_brand"
}
df_attr_laptop_overlap_longest = df_attr_laptop_overlap_longest.rename(columns=columns_dict)
df_attr_laptop_overlap_longest.head()

,type,brand,os,RAM,screen_size,graphic_card_brand,hard_drive_type,HDD_size,SSD_size,processor_model,processor_brand
microcart_0161,None,None,None,1gb,None,None,None,None,8gb,None,None
sdicraft_0604,None,lenovo,None,16gb,None,None,None,16gb,16gb,None,None
bhphotovideo_0001,None,None,None,32gb,None,None,None,32gb,32gb,None,None
truecurate_0345,None,lenovo,None,4gb,None,None,None,32gb,4gb,None,None
macmall_0344,notebook,None,None,32gb,None,None,None,32gb,32gb,None,mediatek


###overlap_shortest for laptop description

In [0]:
offer_id=laptop_description_data['productid']
title=laptop_description_data['description']
list_attr = laptop_list_attr
rd=Laptop_Overlap_shortest(title,offer_id,laptop_list_attr,laptop_dict)
laptop_description_item=rd.get_attribute()
laptop_description_item

df_attr_laptop_descr_overlap_shortest=pd.DataFrame.from_dict(laptop_description_item,orient='index')
columns_dict={
    0:"type",
    1:"brand",
    2:"os",
    3:"RAM",
    4:"screen_size",
    5:"graphic_card_brand",
    6:"hard_drive_type",
    7:"HDD_size",
    8:"SSD_size",
    9:"processor_model",
    10:"processor_brand"
}
df_attr_laptop_descr_overlap_shortest = df_attr_laptop_descr_overlap_shortest.rename(columns=columns_dict)
df_attr_laptop_descr_overlap_shortest.head()

,type,brand,os,RAM,screen_size,graphic_card_brand,hard_drive_type,HDD_size,SSD_size,processor_model,processor_brand
microcart_0161,None,None,None,None,None,None,None,None,None,None,None
sdicraft_0604,None,lenovo,None,2gb,None,None,None,128gb,2gb,None,None
bhphotovideo_0001,None,None,None,None,None,None,None,None,None,None,None
truecurate_0345,None,None,None,None,None,None,None,None,None,None,None
macmall_0344,None,None,None,4gb,None,None,None,32gb,4gb,None,mediatek


###overlap_longest for laptop description

In [0]:
offer_id=laptop_description_data['productid']
title=laptop_description_data['description']
list_attr = laptop_list_attr
rd=Laptop_Overlap_longest(title,offer_id,laptop_list_attr,laptop_dict)
laptop_description_item=rd.get_attribute()
laptop_description_item

df_attr_laptop_descr_overlap_longest=pd.DataFrame.from_dict(laptop_description_item,orient='index')
columns_dict={
    0:"type",
    1:"brand",
    2:"os",
    3:"RAM",
    4:"screen_size",
    5:"graphic_card_brand",
    6:"hard_drive_type",
    7:"HDD_size",
    8:"SSD_size",
    9:"processor_model",
    10:"processor_brand"
}
df_attr_laptop_descr_overlap_longest = df_attr_laptop_descr_overlap_longest.rename(columns=columns_dict)
df_attr_laptop_descr_overlap_longest.head()

,type,brand,os,RAM,screen_size,graphic_card_brand,hard_drive_type,HDD_size,SSD_size,processor_model,processor_brand
microcart_0161,None,None,None,None,None,None,None,None,None,None,None
sdicraft_0604,None,lenovo,None,2gb,None,None,None,128gb,2gb,None,None
bhphotovideo_0001,None,None,None,None,None,None,None,None,None,None,None
truecurate_0345,None,None,None,None,None,None,None,None,None,None,None
macmall_0344,None,None,None,4gb,None,None,None,32gb,4gb,None,mediatek


##Analysis of feature distribution


In [0]:
print('df_attr_dogfood_overlap_shortest')
df_attr_dogfood_overlap_shortest.describe()

df_attr_dogfood_overlap_shortest


,brand,lifestage,breed_size,food_type,flavour,weight,breed,price
count,2962,1647,858,1498,1071,206,161,0.0
unique,13,4,5,4,19,11,7,0.0
top,royal canin,adult,small,dry,chicken,3kg,retriever,NaN
freq,1245,1084,369,1265,482,70,34,NaN


In [0]:
print('df_attr_dogfood_overlap_longest')
df_attr_dogfood_overlap_longest.describe()

df_attr_dogfood_overlap_longest


,brand,lifestage,breed_size,food_type,flavour,weight,breed,price
count,2962,1647,858,1498,1071,206,161,0.0
unique,13,4,5,4,19,11,7,0.0
top,royal canin,adult,small,dry,chicken,3kg,golden retriever,NaN
freq,1245,1084,369,1265,482,70,33,NaN


In [0]:
print('df_attr_dogfood_descr_overlap_shortest')
df_attr_dogfood_descr_overlap_shortest.describe()

df_attr_dogfood_descr_overlap_shortest


,brand,lifestage,breed_size,food_type,flavour,weight,breed,price
count,2321,1812,1124,1543,2273,265,161,0.0
unique,19,4,5,8,18,16,8,0.0
top,royal canin,adult,small,dry,chicken,10 kg,retriever,NaN
freq,727,1359,570,1007,481,49,36,NaN


In [0]:
df_attr_dogfood_descr_overlap_longest.describe()

,brand,lifestage,breed_size,food_type,flavour,weight,breed,7
count,2321,1812,1124,1543,2273,265,161,0.0
unique,19,4,5,8,18,16,8,0.0
top,royal canin,adult,small,dry,chicken,10 kg,retriever,NaN
freq,727,1359,570,1007,481,49,36,NaN


In [0]:
df_attr_laptop_overlap_shortest.describe()

In [0]:
df_attr_laptop_overlap_longest.describe()

In [0]:
df_attr_laptop_descr_overlap_shortest.describe()

In [0]:
df_attr_laptop_descr_overlap_longest.describe()

##Analysis of feature extraction quality

###Quality Evaluation


In [0]:
# loading the gold standard which contains 
# lade gold stnadrad als csv!
# lade mapping seite von offerId und html-adresse
# speicher im data frame 

#nehme vom ersten eintrag die offerid 
#vergleiche mit 

# access the information source used to create the goldstandard

# run coding on source

# calculate falsepositive, 

# calculate recall,

SyntaxError: ignored

###Quantity Evaluation

###Location Evaluation
This part is important to know where the information can be found

##save / store extracted value in csv

In [0]:
df_attr_dogfood_overlap_shortest.to_csv("df_attr_dogfood_overlap_shortest.csv")
files.download("df_attr_dogfood_overlap_shortest.csv")
df_attr_dogfood_overlap_longest.to_csv("df_attr_dogfood_overlap_longest.csv")
files.download("df_attr_dogfood_overlap_longest.csv")
df_attr_dogfood_descr_overlap_shortest.to_csv("df_attr_dogfood_descr_overlap_shortest.csv")
files.download("df_attr_dogfood_descr_overlap_shortest.csv") 
df_attr_dogfood_descr_overlap_longest.to_csv("df_attr_dogfood_descr_overlap_longest.csv")
files.download("df_attr_dogfood_descr_overlap_longest.csv")
df_attr_laptop_overlap_shortest.to_csv("df_attr_laptop_overlap_shortest.csv")
files.download("df_attr_laptop_overlap_shortest.csv")
df_attr_laptop_overlap_longest.to_csv("df_attr_laptop_overlap_longest.csv")
files.download("df_attr_laptop_overlap_longest.csv")
df_attr_laptop_descr_overlap_shortest.to_csv("df_attr_laptop_descr_overlap_shortest.csv")
files.download("df_attr_laptop_descr_overlap_shortest.csv")
df_attr_laptop_descr_overlap_longest.to_csv("df_attr_laptop_descr_overlap_longest.csv")
files.download("df_attr_laptop_descr_overlap_longest.csv")

NameError: ignored